# Step1: Source injection

In [1]:
import lib.butler as btl
import lib.tools as tl
import lib.stamp as stp
import lib.inject as inj
import lib.visual as vis

from astropy.io import fits
import os
from pathlib import Path

import random
import json

In [2]:
#%matplotlib inline

In [3]:
folder = Path(tl.CATALOG_FOLDER)
folder.mkdir(parents=True, exist_ok=True)

folder = Path(tl.FIG_FOLDER)
folder.mkdir(parents=True, exist_ok=True)

Begin with ECDFS (center). 

In [4]:
ra_cen = 53.076
dec_cen = -28.110
band = 'i'

Define the grid for injection. It is based on RA/DEC rather than X/Y pixel coordinates. 

Also, the grid is near the (ECDFS) field center, not the image center.

In [5]:
width_arcmin = 10 #2
num_side = 40 #10

inj_radec = inj.make_grid(ra_cen, dec_cen, width_arcmin, num_side)

gap_arcsec:  8.615384615384615


## Visit Image Injection

In [6]:
dataset_refs = btl.get_visit_dataset_refs(ra_cen, dec_cen, band)
print(len(dataset_refs))
print(dataset_refs[0])

148
visit_image@{instrument: 'LSSTComCam', detector: 0, visit: 2024110800245, band: 'i', day_obs: 20241108, physical_filter: 'i_06'} [sc=ExposureF] (run=LSSTComCam/runs/DRP/DP1/DM-53601 id=019b1941-8acf-7980-8487-d82d7c24a5af)


### Select one visit

In [7]:
visit_index = random.randint(0, len(dataset_refs)-1) #0 
visit_image = btl.get_visit_image(dataset_refs, visit_index)
visit_rotation_angle = tl.get_visit_rotation_angle(visit_image)
print(visit_rotation_angle, "deg")

115.25834308666 deg


In [8]:
dataId = dataset_refs[visit_index].dataId
visit = dataId['visit']
detector = dataId['detector']
visit_tag = f"visit_{visit}_{detector}"

### Choose a stamp (and rotate)

In [9]:
system_index_list = [1,4]
time_index_list = range(5) #[4]
stamp_folder = tl.STAMP_FOLDER

Make the list of stamps to inject onto the grid.

Rotate the stamp before injecting into the visit image.

In [10]:
tot_num = num_side ** 2
stamp_filename_list = []
coadd_stamp_filename_list = []
stamp_mag_list = []
coadd_stamp_mag_list = []

for ind in range(tot_num):
    if ind%100==0:
        print(f"Checking location No. {ind}...")

    system_index = random.choice(system_index_list)
    
    time_index = random.choice(time_index_list)

    system_tag = f"system_{system_index}"

    stamp_tag = f"{system_tag}_{time_index}"
    wcs_stamp_filename = f"{stamp_folder}/{stamp_tag}_wcs.fits"
    rot_wcs_stamp_filename = wcs_stamp_filename.replace(".fits", f"_rot_{visit_tag}.fits")
    stamp_filename_list.append(rot_wcs_stamp_filename)
    
    stamp_mag = fits.getval(wcs_stamp_filename, "total_mag")
    #print("stamp_mag: ", stamp_mag)
    stamp_mag_list.append(stamp_mag)
    
    path = Path(rot_wcs_stamp_filename)
    if not path.exists():
        stp.make_rotated_stamp(visit_rotation_angle, wcs_stamp_filename, rot_wcs_stamp_filename)
        
    
    coadd_stamp_tag = f"{system_tag}_coadd"
    wcs_coadd_stamp_filename = f"{stamp_folder}/{coadd_stamp_tag}_wcs.fits"
    coadd_stamp_filename_list.append(wcs_coadd_stamp_filename)
    
    coadd_stamp_mag = fits.getval(wcs_coadd_stamp_filename, "total_mag")
    #print("coadd_stamp_mag: ", coadd_stamp_mag)
    coadd_stamp_mag_list.append(coadd_stamp_mag)

Checking location No. 0...


Checking location No. 100...
Checking location No. 200...
Checking location No. 300...
Checking location No. 400...
Checking location No. 500...
Checking location No. 600...
Checking location No. 700...
Checking location No. 800...
Checking location No. 900...
Checking location No. 1000...
Checking location No. 1100...
Checking location No. 1200...
Checking location No. 1300...
Checking location No. 1400...
Checking location No. 1500...


### Inject the (rotated) stamp to a visit image

Inject onto the grid.

In [11]:
#inj_radec

In [12]:
inj_catalog_visit = inj.make_inj_catalog_visit(
                                visit_image, 
                                stamp_mag_list, 
                                stamp_filename_list, 
                                inj_radec
                        )

injected_visit_image, injected_visit_catalog = inj.visit_inject_stamp(visit_image, inj_catalog_visit)

In [13]:
#injected_visit_catalog

In [14]:
#injected_visit_catalog

In [15]:
visit_image.writeFits(f"{tl.FIG_FOLDER}/{visit_tag}.fits")

injected_visit_image.writeFits(f"{tl.FIG_FOLDER}/injected_{visit_tag}.fits")

### Check the image with cutout

Make a cutout at the central region.

In [16]:
vis.plot_dual(visit_image, injected_visit_image, inj_radec, visit_tag)

## Template injection

In [17]:
dataset_refs = btl.get_template_dataset_refs(ra_cen, dec_cen, band)
print(len(dataset_refs))
print(dataset_refs[0])

2
template_coadd@{band: 'i', skymap: 'lsst_cells_v1', tract: 5063, patch: 14} [sc=ExposureF] (run=LSSTComCam/runs/DRP/DP1/DM-51335 id=014f2da9-c3ff-495f-83ef-e1a1ab666231)


### Select one template

In [18]:
template_index = random.randint(0, len(dataset_refs)-1) #0 
template_image = btl.get_template_image(dataset_refs, template_index)

In [19]:
dataId = dataset_refs[template_index].dataId
tract = dataId['tract']
patch = dataId['patch']
#band = dataId['band'] 
template_tag = f"template_{tract}_{patch}_{band}"

Follow similar procedure to the visit images, but note coadd patches do not need the rotation.

### Inject the (unrotated) stamp to a template image

Inject onto the grid.

In [20]:
inj_catalog_template = inj.make_inj_catalog_template(
                                template_image,
                                coadd_stamp_mag_list,
                                coadd_stamp_filename_list,
                                inj_radec
                            )

injected_template_image, injected_template_catalog = inj.template_inject_stamp(template_image, inj_catalog_template)

In [21]:
#injected_template_catalog

In [22]:
template_image.writeFits(f"{tl.FIG_FOLDER}/{template_tag}.fits")

injected_template_image.writeFits(f"{tl.FIG_FOLDER}/injected_{template_tag}.fits")

### Check the image with cutout

Make a cutout at the central region.

In [23]:
vis.plot_dual(template_image, injected_template_image, inj_radec, template_tag, )

In [24]:
print(visit_tag)
print(template_tag)

visit_2024120500062_4
template_5063_14_i


In [25]:
data = {
    "visit": visit,
    "detector": detector,
    "tract": tract,
    "patch": patch,
    "band": band,
}

with open("numbers.json", "w") as f:
    json.dump(data, f)

